In [4]:
import numpy as np
import pandas as pd

from litedl.models import Sequential
from litedl.layers import Affine, SoftmaxWithLoss, ReLU
from litedl.optimizers import SGD
from litedl.utils import data_split, OneHotEncoder

# 학습 하이퍼 파라미터 설정
epochs = 300
batch_size = 20

# 붓꽃 데이터셋 불러오기
dataset = pd.read_csv('dataset/Iris.csv')
dataset.drop('Id', axis=1, inplace=True)

# 피처와 라벨 데이터 나누기
features = dataset.iloc[:, :-1].values
labels = dataset.iloc[:, -1].values.reshape(-1, 1)

# 라벨 데이터를 원 핫 인코딩으로 표현하고 섞기
encoder = OneHotEncoder()
encoder.fit(labels)
labels = encoder.transform(labels)

# 데이터 섞기
indices = np.arange(features.shape[0])
np.random.shuffle(indices)
features = features[indices]
labels = labels[indices]

# 학습용 데이터와 테스트용 데이터로 나누기
train_x, test_x = data_split(features, 0.2)
train_y, test_y = data_split(labels, 0.2)

In [5]:
# 모델과 계층 선언
model = Sequential()
affine1 = Affine(input_size=4, output_size=5)
relu = ReLU()
affine2 = Affine(input_size=5, output_size=3)
loss_fn = SoftmaxWithLoss()
optimizer = SGD(lr=0.1)

# 계층을 쌓아 모델을 구축
model.add_layer(affine1)
model.add_layer(relu)
model.add_layer(affine2)
model.add_loss_layer(loss_fn)

# 모델 학습
for epoch in range(epochs):
    loss = model.forward(train_x, train_y)
    model.backward()
    model.step(optimizer=optimizer)
    print(f'epoch {epoch} loss: {loss}')

epoch 0 loss: 1.4037853723007074
epoch 1 loss: 1.2958004228761022
epoch 2 loss: 1.2211047910942174
epoch 3 loss: 1.1682042773085162
epoch 4 loss: 1.1338195880579103
epoch 5 loss: 1.1120328143845777
epoch 6 loss: 1.096037530570697
epoch 7 loss: 1.0812963158537883
epoch 8 loss: 1.0658226436630893
epoch 9 loss: 1.0489028055513103
epoch 10 loss: 1.0302425057894742
epoch 11 loss: 1.009692132566416
epoch 12 loss: 0.9871985164770469
epoch 13 loss: 0.9628142195149852
epoch 14 loss: 0.9367129358854356
epoch 15 loss: 0.9092019589102255
epoch 16 loss: 0.8807506381121777
epoch 17 loss: 0.8518976236237925
epoch 18 loss: 0.8231945808305751
epoch 19 loss: 0.7951792431315988
epoch 20 loss: 0.7685188090175825
epoch 21 loss: 0.7436526180572713
epoch 22 loss: 0.7210916051706865
epoch 23 loss: 0.7011346031413808
epoch 24 loss: 0.6837963650172062
epoch 25 loss: 0.668689452594289
epoch 26 loss: 0.6555806664618568
epoch 27 loss: 0.644021619609268
epoch 28 loss: 0.6338680984003184
epoch 29 loss: 0.62475667247

In [6]:
accuracy = np.sum(np.argmax(model.predict(test_x), axis=1) == np.argmax(test_y, axis=1)) / test_x.shape[0]

print(f'Accuracy: {accuracy}')

Accuracy: 0.9
